## Setup  

In this notebook we use a **Neural network based design representation** for the topology optimisation (TO) of a double pipe. 

In particular, instead of designing based on parameterizing with the element densities, we parameterize using the weights of the neural newtork $\mathbf{w}$.
For the forward Navier–Stokes–Brinkman implementation, refer to
`solve_fluid.ipynb`.

\[
\begin{aligned}
\min_{\mathbf{w}}\;\; & J
      = \int_{\Omega}
        \underbrace{\tfrac12(\,\mu\,\bigl(\nabla\mathbf u
        +\nabla\mathbf u^{\mathsf T}\bigr)\!:\!
        \bigl(\nabla\mathbf u+\nabla\mathbf u^{\mathsf T}\bigr) +\alpha(\mathbf{w})\,\mathbf u\cdot\mathbf u)}_{\text{viscous dissipation}}
        \;d\Omega                                             \\[4pt]
\text{s.\,t.}\;\;
& \mathbf R(\mathbf{w}, \mathbf u, p) = \mathbf 0 \qquad\; \text{(steady NS–Brinkman)}\\[2pt]
& g(\mathbf{w})=1-\frac{\displaystyle
             \sum_e \gamma_e(\mathbf{w})\,v_e}{V^{\ast}} \le 0        \qquad
  \text{(volume / porosity constraint)}
\end{aligned}
\]

where  
* **Design variable** $\mathbf{w}$: The weights of the neural network. The coordinates of the domain are the inputs of the network and the element fill ratio $\gamma$ are the outputs.
* **Design field**  $\gamma_e \in [0,1]$ — element fill ratio (0 = fluid, 1 = solid) obtained by querying the element coordinate centers $\gamma_e = NN(\mathbf{x}_e; \mathbf{w})$  
* **State variables**  $\mathbf u$ (velocity) and $p$ (pressure)  
* **Residual**  $\mathbf R(\gamma,\mathbf u,p)=\mathbf 0$ — discrete steady NS–Brinkman equations  
* $v_e$ — element area, $V^{\ast}$ — prescribed minimum solid/material fraction  

The objective $J$ minimizes **total viscous dissipation**
(equivalent to pressure drop for a fixed flow rate), a common goal in
fluid-device design.  
To target another metric (e.g. reverse flow or minimize flow drag)
simply replace the definition of $J$.

### Double Pipe Boundary Conditions
To illustrate fluid TO, we consider a simple double pipe problem, as shown in the figure below:

<img src="../../../figures/double_pipe.png" alt="Channel" style="width: 500px;"/>

### Design parametrisation  

We adopt the neural network based design parameterization as proposed in:


Chandrasekhar, Aaditya, and Krishnan Suresh. "TOuNN: Topology optimization using neural
networks." Structural and Multidisciplinary Optimization 63, no. 3 (2021): 1135-1149.


where the weights of the network acts as the design variable

### Optimiser  

We employ the **Log barrier scheme** where we formulate the constrained optimization 
problem as an equivalent loss.
For more details see `constrained_loss.py`

Gradients $\partial L/\partial \mathbf{w}$ are produced automatically with JAX’s AD.

NOTE: We follow a convention similar to those found in the solid mechanics literature of Topology optimization where the solids are attributed with a pseudodensity of 1 and voids/fluids with a pseudodensity of 0. This is in contrast to literature in fluid mechanics where the vice-versa is often use. 

In [ ]:
import functools
import jax.numpy as jnp
import jax
from jax.typing import ArrayLike
from flax import nnx
import optax
import matplotlib.pyplot as plt

import toflux.src.geometry as _geom
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.utils as _utils
import toflux.src.bc as _bc
import toflux.experiments.fluid.tounn.network as _network
import toflux.src.fe_fluid as _fea
import toflux.src.solver as _solv
import toflux.src.constrained_loss as _cons_loss
import toflux.src.viz as _viz

jax.config.update("jax_enable_x64", True)
plt.rcParams.update(_viz.high_res_plot_settings)

_Ext = _utils.Extent
_Field = _fea.FluidField
_cmap = _viz.fluid_cmap

# Define Geometry and Mesh 

Below we build the **double_pipe** benchmark commonly used in fluid TO.

#### Geometry  
A simple double-pipe is described in a json and passed as the geometry. This geometry is passed to the mesher.

#### Discretisation  
* The domain $(\Omega)$ is partitioned into **bilinear quadrilateral (Q1) elements**.  
* Both primary fields—velocity $(\mathbf u=(u_x,u_y))$ **and** pressure (p\)—are interpolated with the same Q1 shape functions (equal-order Q1/Q1 formulation).

#### Numerical integration  
Element contributions (mass, convection, brinkman, viscous and stability terms) are evaluated with a $(2 \times 2)$ Gauss quadrature.

This mesh serves as the baseline for the finite-element Navier–Stokes solve and for any subsequent density-based topology-optimisation runs.

In [ ]:
geom = _geom.BrepGeometry(brep_file="toflux/brep/double_pipe_long.json")

mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=150,
  nely_desired=100,
  dofs_per_node=3,
  gauss_order=2,
)

_viz.plot_brep(geom)

### Material  

For this fluid demonstration we keep the parameters dimensionless:

| Symbol | Meaning | Value |
|--------|---------|-------|
| $(\rho)$ | Mass density | **1** |
| $(\mu)$ | Dynamic viscosity | **1** |

We also prepare the *inverse permeability* field used in the Brinkman term.  
A RAMP interpolation converts the material fraction $(\gamma\in[0,1])$ into the damping coefficient $(\alpha)$. In the subsequent topology optimization the brinkman penalty value damps fluid flow in solid.

In [ ]:
mat_params = _mat.FluidMaterial(
  mass_density=1.0,
  dynamic_viscosity=1.0,
)
min_mat_frac = 0.666
mat_frac = min_mat_frac * jnp.ones((mesh.num_elems,))

min_inv_permeability = _mat.brinkman_bound(
  mat_params.dynamic_viscosity, 100.0 * mesh.bounding_box.lx
)
max_inv_permeability = _mat.brinkman_bound(
  mat_params.dynamic_viscosity, 1.0e-2 * mesh.bounding_box.lx
)
init_inv_permeability = _mat.brinkman_bound(
  mat_params.dynamic_viscosity, 1.0e-1 * mesh.bounding_box.lx
)


inv_permeability_ext = _utils.Extent(min=min_inv_permeability, max=max_inv_permeability)
init_ramp_penalty = _mat.calculate_initial_ramp_penalty(
  inv_permeability_ext, init_inv_permeability, min_mat_frac
)

# Boundray Conditions


| Region | Type | Imposed values |
|--------|------|----------------|
| **Inlet top and bottom(left vertical edge)** | Dirichlet | $(u = parabolic)$, $(v = 0)$ |
| **Outlet top and bottom (right vertical edge)** | Dirichlet v, p | \(p = 0\), \(v = 0\) |
| **Top wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |
| **Bottom wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |

**Characteristic velocity**

The inlet velocity is chosen to give a Reynolds number of 1:

$$
U_c \;=\; \frac{\mathrm{Re}\,\nu}{H},
\qquad
\text{Re} = 1,\;
\quad
\nu = \frac{\mu}{\rho},
\quad
H = \text{characteristic length}.
$$

In [ ]:
inlet_fraction = 1.0 / 6.0
char_length = inlet_fraction * mesh.bounding_box.ly
reynolds_num = 0.1667
char_velocity = reynolds_num * mat_params.kinematic_viscosity / (char_length)

In [ ]:
face_tol = mesh.elem_size[0] * 0.5

# inlet-top condition
inlet_top_faces = _bc.identify_faces(mesh, edges=[geom.edges[3]], tol=face_tol)
n_faces = len(inlet_top_faces)
x_nodes = jnp.linspace(-1.0, 1.0, n_faces + 1)
u_nodes = char_velocity * (1.0 - x_nodes**2)
face_node_vals = jnp.stack([u_nodes[1:], u_nodes[:-1]], axis=1)
u_vel = (_Field.U_VEL, face_node_vals)
v_vel = (_Field.V_VEL, jnp.zeros_like(face_node_vals))
inlet_top_face_val = [u_vel, v_vel]

# inlet bottom condition
inlet_btm_faces = _bc.identify_faces(mesh, edges=[geom.edges[1]], tol=face_tol)
n_faces = len(inlet_btm_faces)
x_nodes = jnp.linspace(-1.0, 1.0, n_faces + 1)
u_nodes = char_velocity * (1.0 - x_nodes**2)
face_node_vals = jnp.stack([u_nodes[1:], u_nodes[:-1]], axis=1)
u_vel = (_Field.U_VEL, face_node_vals)
v_vel = (_Field.V_VEL, jnp.zeros_like(face_node_vals))
inlet_btm_face_val = [u_vel, v_vel]

# wall condition
wall_faces = _bc.identify_faces(
  mesh,
  edges=[
    geom.edges[0],
    geom.edges[2],
    geom.edges[4],
    geom.edges[6],
    geom.edges[8],
    geom.edges[10],
    geom.edges[5],
    geom.edges[11],
  ],
  tol=face_tol,
)
n = len(wall_faces)
u_vel = (_Field.U_VEL, jnp.zeros(n))
v_vel = (_Field.V_VEL, jnp.zeros(n))
wall_face_val = [u_vel, v_vel]

# out flow condition
outflow_faces = _bc.identify_faces(
  mesh, edges=[geom.edges[7], geom.edges[9]], tol=face_tol
)
n = len(outflow_faces)
v_vel = (_Field.V_VEL, jnp.zeros(n))
pres = (_Field.PRESSURE, jnp.zeros(n))
outflow_face_val = [v_vel, pres]


inlet_top_bc = _bc.DirichletBC(
  elem_faces=inlet_top_faces, values=inlet_top_face_val, name="Inlet Top"
)
inlet_btm_bc = _bc.DirichletBC(
  elem_faces=inlet_btm_faces, values=inlet_btm_face_val, name="Inlet Bottom"
)
outflow_bc = _bc.DirichletBC(
  elem_faces=outflow_faces, values=outflow_face_val, name="Outflow"
)
wall_bc = _bc.DirichletBC(elem_faces=wall_faces, values=wall_face_val, name="Wall")


bcs_list = [
  wall_bc,
  inlet_top_bc,
  inlet_btm_bc,
  outflow_bc,
]

bc = _bc.process_boundary_conditions(bcs_list, mesh)

# Solver


The incompressible Navier–Stokes–Brinkman system is **intrinsically non-linear** because of the convective term .
Our solver therefore employs a *damped/modified Newton–Raphson* loop. For more details see 'solver.py'.

# Loss

We convert the constrained optimization problem into an unvonstrained loss using the 
log barrier scheme. 

In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.PETSC,
    "rtol": 1.0e-5,
    "petsc_solver": {},
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-6},
}
flow_solver = _fea.FluidSolver(mesh, bc, mat_params, solver_settings=solver_settings)

loss_params = _cons_loss.LogBarrierParams(t0=3.0, mu=1.04)

## Network

We define a simple fully connected neural network that maps the spatial coordinates
to the material fraction. Furthermore, we impose a symmetry projection that ensures that the obtained designs are indeed symmeteric. Finally, we project the coordinates from the Euclidean space to the Fourier space before passing them through the network.
This ensures that the network is able to capture finer resolutions in the design. For more details see:

Chandrasekhar, Aaditya, and Krishnan Suresh. "Approximate length scale filter in topology optimization using fourier enhanced neural networks." Computer-Aided Design 150 (2022): 103277.

In [ ]:
symm_map = _network.Symmetry(sym_xz_coord=mesh.bounding_box.y.center)


fourier_proj = _network.FourierProjection(
  num_input_dim=2,
  num_terms=100,
  min_radius=0.25,
  max_radius=5.0,
)

topnet = _network.TopNet(
  num_neurons=[2 * fourier_proj.num_terms, 20, 1],
  rngs=nnx.Rngs(0),
  use_batch_norm=False,
)

# Optimize
Here, we define the optimization loop. We use the Adam optimizer. 

The NN is first queried for the material fraction at the spatial coordinates.
Then, the obtained material distribution is passed to the solver to obtain the pressure and velocity field. Then we compute the dissipated power. Similarly, we obtain the constraint values using the obtained material distribution. A combined loss is then obtained using the log-barrier formulation. Finally, we utilize the automatic differentiation capabilities of JAX to obtain the gradients of the loss with respect to the weights of the network.


 The optimization flowchart is illustrated below:


<img src="../../../figures/flowchart_tounn.png" alt="flowchart" style="width: 800px;"/>

In [ ]:
@functools.partial(nnx.jit)
def loss_fn(
  net: _network.TopNet,
  mesh_xy: jnp.ndarray,
  epoch: int,
  max_vol_frac: float,
  obj_0: float,
  ramp_penalty: float,
  press_vel_guess: ArrayLike,
):
  mat_frac = jax.nn.sigmoid(net(mesh_xy)).reshape((-1,))
  brinkman_penalty = _mat.compute_ramp_interpolation(
    prop=mat_frac,
    ramp_penalty=ramp_penalty,
    prop_ext=inv_permeability_ext,
    mode="convex"
  )
  press_vel = _solv.modified_newton_raphson_solve(
    flow_solver, press_vel_guess, brinkman_penalty
  )
  obj_args = (
    brinkman_penalty,
    press_vel[mesh.elem_dof_mat],
    mesh.elem_node_coords,
  )
  elem_diss_pow = jax.vmap(flow_solver.compute_elem_dissipated_power)(*obj_args)
  dissipated_power = jnp.sum(elem_diss_pow)

  volcons = jnp.array([1.0 - (mat_frac.mean() / max_vol_frac)])

  loss = _cons_loss.combined_loss(
    dissipated_power / obj_0,
    volcons,
    [_cons_loss.ConstraintTypes.INEQUALITY],
    [loss_params],
    epoch,
  )

  return loss, (
    mat_frac,
    press_vel,
    dissipated_power,
    volcons,
  )

In [ ]:
def optimize_design(
  net: _network.TopNet,
  max_vol_frac: float = 0.6,
  lr: float = 1.0e-2,
  max_iter: int = 100,
):
  mesh_xy = symm_map.apply(jnp.array(mesh.elem_centers))
  mesh_xy = fourier_proj.apply(mesh_xy)

  opt = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.adam(lr),
  )
  optimizer = nnx.Optimizer(topnet, opt)

  iter = 0
  obj_0 = 1.0
  convg_hist = {"diss_pow": [], "vol_frac": []}

  press_vel = jnp.zeros((mesh.num_dofs,))
  press_vel = press_vel.at[bc["fixed_dofs"]].set(bc["dirichlet_values"])

  for iter in range(max_iter):
    cont_param = min(20.0, 1.0 + 19.0 * (iter / 75.0) ** 2)
    ramp_penalty = init_ramp_penalty / cont_param
    (loss, (density, press_vel, diss_pow, vol_frac)), grad_loss = nnx.value_and_grad(
      loss_fn, has_aux=True
    )(net, mesh_xy, iter, max_vol_frac, obj_0, ramp_penalty, press_vel)

    press_vel = jax.lax.stop_gradient(press_vel)
    press_vel = press_vel.at[bc["fixed_dofs"]].set(bc["dirichlet_values"])

    optimizer.update(grad_loss)

    # save and print
    print(f"iter {iter}, loss {loss:.2E}")
    print(f"diss_pow {diss_pow:.2E}, vol_frac {vol_frac[0]:.2F}")

    convg_hist["diss_pow"].append(diss_pow)
    convg_hist["vol_frac"].append(vol_frac)

    # renormalization and penalty factor
    if iter == 0 or iter == 10:
      obj_0 = jax.lax.stop_gradient(diss_pow)

    # plotting
    if iter % 10 == 0:
      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(
        mesh=mesh,
        field=density.reshape(-1),
        ax=ax,
        colorbar=False,
        cmap=_cmap,
        val_range=(0.0, 1.0),
      )

      ax.set_aspect("equal")
      ax.spines[["top", "right", "left", "bottom"]].set_visible(False)
      ax.set_xticks([])
      ax.set_yticks([])
      plt.show()
      plt.pause(1e-6)

  return convg_hist

In [ ]:
convg_hist = optimize_design(topnet, max_vol_frac=0.67)